In [14]:
import re
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

# Load your dataset
data = pd.read_csv("datasets/weekly_scoring.csv")

# filter only QB
data = data[data['POS'] == 'qb']

# Assuming you have a date column and columns to predict for QB only
date_column = "DATE"
columns_to_predict = ['PASSING CMP', 'PASSING ATT', 'PASSING PCT', 'PASSING YDS', 'PASSING Y/A', 'PASSING TD', 'PASSING INT',
        'PASSING SACKS', 'RUSHING ATT', 'RUSHING YDS', 'RUSHING TD', 'MISC FL', 'WEEK', 'AVG_FPTS', 'MAX_FPTS', 'MIN_FPTS', 'VAR_FPTS']


# Sort the data by the date column
data[date_column] = pd.to_datetime(data[date_column])
data = data.sort_values(by=date_column)

# Extract the relevant columns for training
training_data = data[columns_to_predict].values

# Impute missing values for training_data
# Handling missing values in numeric columns using SimpleImputer
imputer = SimpleImputer(strategy='mean')  # You can change the strategy as needed
training_data = imputer.fit_transform(training_data)

# Normalize the data using Min-Max scaling
scaler = MinMaxScaler(feature_range=(0, 1))
training_data_scaled = scaler.fit_transform(training_data)

# Define a function to create LSTM datasets
def create_lstm_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back)])
        Y.append(dataset[i + look_back])
    return np.array(X), np.array(Y)

# Set the number of time steps to look back
look_back = 25  # You can adjust this value based on the characteristics of your data

# Create the LSTM dataset
X, Y = create_lstm_dataset(training_data_scaled, look_back)

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
Y_train, Y_test = Y[:train_size], Y[train_size:]

# Reshape the input data for LSTM (samples, time steps, features)
X_train = np.reshape(X_train, (X_train.shape[0], look_back, X_train.shape[2]))
X_test = np.reshape(X_test, (X_test.shape[0], look_back, X_test.shape[2]))

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=len(columns_to_predict)))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_test, Y_test), verbose=1)

# Make predictions on the test set
predictions = model.predict(X_test)

# Rescale the predictions to the original scale
predictions_rescaled = scaler.inverse_transform(predictions)
Y_test_rescaled = scaler.inverse_transform(Y_test)

# Evaluate the model
mse = mean_squared_error(Y_test_rescaled, predictions_rescaled)
print(f'Mean Squared Error: {mse}')

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(Y_test_rescaled, predictions_rescaled))
print(f"RMSE: {rmse}")

# Mean Absolute Error (MAE)
mae = mean_absolute_error(Y_test_rescaled, predictions_rescaled)
print(f"MAE: {mae}")

# R-squared
r_squared = r2_score(Y_test_rescaled, predictions_rescaled)
print(f"R-squared: {r_squared}")

#Plot the results for one of the columns
# column_index = 0  # You can change this index based on the column you want to visualize
# plt.plot(Y_test_rescaled[:, column_index], label='Actual')
# plt.plot(predictions_rescaled[:, column_index], label='Predicted')
# plt.legend()
# plt.show()

# Create a DataFrame to store the results
results_list = []

# Create a DataFrame with 'PLAYER' and predicted values
result_df = pd.DataFrame({'PLAYER': data['PLAYER'].iloc[train_size + look_back:], 'Predicted_FPTS': predictions_rescaled[:, 0]})

# Group by 'PLAYER' and calculate the average predicted FPTS
result_df = result_df.groupby('PLAYER').mean().reset_index()
result_df = result_df.sort_values(by='Predicted_FPTS', ascending=False)

# Add the 'TEAM' column if needed (replace with your own logic)
pattern = r'\((.*?)\)'
result_df['TEAM'] = result_df['PLAYER'].apply(lambda x: re.search(pattern, x).group(1) if re.search(pattern, x) else pd.NA)

# Keep only the first unique occurrence of any value in the 'TEAM' column
result_df = result_df.drop_duplicates(subset='TEAM')

# Append the results to the list
results_list.append(result_df)

# Combine the results from all folds
final_results_df = pd.concat(results_list, ignore_index=True)

# Remove any player with the team equal to 'FA'
final_results_df = final_results_df.query("TEAM != 'FA'")


# # Save the results to a CSV file
# file_name = "predictions/LSTM_predictions.csv"
# final_results_df.to_csv(file_name, index=False)

final_results_df.head(10)

Epoch 1/100
64/64 [==============================] - 1s 7ms/step - loss: 0.0467 - val_loss: 0.0253
Epoch 2/100
64/64 [==============================] - 0s 4ms/step - loss: 0.0264 - val_loss: 0.0221
Epoch 3/100
64/64 [==============================] - 0s 4ms/step - loss: 0.0241 - val_loss: 0.0206
Epoch 4/100
64/64 [==============================] - 0s 4ms/step - loss: 0.0231 - val_loss: 0.0201
Epoch 5/100
64/64 [==============================] - 0s 4ms/step - loss: 0.0227 - val_loss: 0.0198
Epoch 6/100
64/64 [==============================] - 0s 4ms/step - loss: 0.0225 - val_loss: 0.0199
Epoch 7/100
64/64 [==============================] - 0s 4ms/step - loss: 0.0224 - val_loss: 0.0193
Epoch 8/100
64/64 [==============================] - 0s 4ms/step - loss: 0.0222 - val_loss: 0.0197
Epoch 9/100
64/64 [==============================] - 0s 4ms/step - loss: 0.0221 - val_loss: 0.0199
Epoch 10/100
64/64 [==============================] - 0s 4ms/step - loss: 0.0220 - val_loss: 0.0199
Epoch 11/

,PLAYER,Predicted_FPTS,TEAM
0,Lamar Jackson (BAL),21.972656,BAL
1,Josh Allen (BUF),21.499369,BUF
2,Joshua Dobbs (MIN),21.477657,MIN
3,C.J. Stroud (HOU),21.376776,HOU
4,Dak Prescott (DAL),21.373249,DAL
5,Jordan Love (GB),21.289032,GB
6,Baker Mayfield (TB),21.210546,TB
7,Will Levis (TEN),19.883373,TEN
8,Russell Wilson (DEN),18.750282,DEN
9,Bryce Young (CAR),18.254723,CAR
